[slides](https://docs.google.com/presentation/d/19K9nVjuSOCrZGM6lmFeEEarTm2xZwDSiZEIzf-Ywr5o/edit?usp=sharing)

[python-ecdsa docs](https://github.com/warner/python-ecdsa)

# Signing our First Message with ECDSA

In [68]:
from ecdsa import SigningKey, SECP256k1

# generate private/public key pairs
private_key = SigningKey.generate(curve=SECP256k1)
public_key = private_key.get_verifying_key()

message = b"orange coin good"
signature = private_key.sign(message)

In [69]:
public_key.verify(signature,message)

True

In [70]:
# try:
# catch BadSignatureError:
#     print("bad")
try:
    public_key.verify(signature, b"number go up")
except:
    print("bad signature")


bad signature


# Defining ECDSACoin

* A coin is just a list of transfers, just like with PNGCoin. 
    * Where transfers were photographs of signatures in PNGCoin, they are ECDSA digital signatures in ECDSACoin
* The `public_key` in the last transfer is who owns the coin
* To spend the coin append a new transfer. Use the public key of the person you are sending to, and sign it using your private key.

In [71]:
class Transfer:
    def __init__(self, signature, public_key):
        self.signature = signature
        self.public_key = public_key
        
class ECDSACoin:
    def __init__(self, transfers):
        self.transfers = transfers

In [72]:
# The usual suspects ... 
# SECP256k1 is a detail about the "magical multiplication" used under the covers

bank_private_key = SigningKey.generate(curve=SECP256k1)
bob_private_key = SigningKey.generate(curve=SECP256k1)
alice_private_key = SigningKey.generate(curve=SECP256k1)

bank_public_key = bank_private_key.get_verifying_key()
bob_public_key = bob_private_key.get_verifying_key()
alice_public_key = alice_private_key.get_verifying_key()

In [73]:
from utils import serialize

def issue(public_key):
    #serialize public key we are issuing to
    message = serialize(public_key)
    #sign message issueing to a given public_key with  our private_key
    signature = bank_private_key.sign(message)
    
    transfer = Transfer(signature=signature,public_key=public_key)
    coin = ECDSACoin([transfer])
    
    return coin

# Validating the First Transfer

In [74]:
def validate(coin):
    transfer = coin.transfers[0]
    message = serialize(transfer.public_key)
    
    bank_public_key.verify(transfer.signature, message)

In [75]:
alice_coin = issue(alice_public_key)

validate(alice_coin)

In [76]:
# create invalid transaction
message = serialize(bob_public_key)

signature = bob_private_key.sign(message)
    
transfer = Transfer(signature=signature,public_key=bob_public_key)

bad_coin = ECDSACoin([transfer])

In [77]:
from ecdsa import BadSignatureError

# validate
try:
    validate(bad_coin)
except BadSignatureError:
    print("fraud alert!")

fraud alert!


# Validating Subsequent Transfers

In [78]:
def transfer_message(previous_signature, next_owner_public_key):
    return serialize({
        "previous_signature": previous_signature,
        "next_owner_public_key": next_owner_public_key
    })


def validate(coin):
    #check the first transfer
    transfer = coin.transfers[0]
    message = serialize(transfer.public_key)
    bank_public_key.verify(transfer.signature, message)
    
    #check the rest of transfers
    previous_transfer = coin.transfers[0]
    for next_transfer in coin.transfers[1:]:
        message = transfer_message(previous_transfer.signature, next_transfer.public_key)
        previous_transfer.public_key.verify(next_transfer.signature, message)
        

In [79]:
def get_owner(coin):
    database = {
        serialize(bob_public_key): "Bob",
        serialize(alice_public_key): "Alice",
        serialize(bank_public_key): "Bank"
    }
    
    public_key = serialize(coin.transfers[-1].public_key)
    
    return database[public_key]

In [80]:
# initial issue to alice
ec_coin = issue(alice_public_key)

print("this coin is owned by", get_owner(ec_coin)) 

# alice transfers to bob
message = transfer_message(ec_coin.transfers[-1].signature, bob_public_key)
alice_to_bob = Transfer(
    signature=alice_private_key.sign(message),
    public_key = bob_public_key
)

ec_coin.transfers.append(alice_to_bob)
print("this coin is owned by", get_owner(ec_coin)) 

# bob transfers to bank
message = transfer_message(ec_coin.transfers[-1].signature, bank_public_key)
bob_to_bank = Transfer(
    signature=bob_private_key.sign(message),
    public_key = bank_public_key
)

ec_coin.transfers.append(bob_to_bank)
print("this coin is owned by", get_owner(ec_coin)) 


this coin is owned by Alice
this coin is owned by Bob
this coin is owned by Bank


# Serialization

In [81]:
from utils import to_disk, from_disk

In [82]:
import os

filename = "coin.ecdsacoin"

print("does this coinfile exist on disk?", os.path.isfile(filename))

does this coinfile exist on disk? True


In [83]:
# write coin to disk
ec_coin = issue(alice_public_key)

to_disk(ec_coin, filename)

In [84]:
print("does this coinfile exist on disk?", os.path.isfile(filename))

does this coinfile exist on disk? True


In [64]:
# read coin to disk

ec_coin = from_disk(filename)
ec_coin

# The Finished Product

[ecdsacoin.py](ecdsacoin.py)

In [21]:
import ecdsacoin

coin = ecdsacoin.issue(alice_public_key)
coin.validate()

alice_to_bob = Transfer(
    signature=alice_private_key.sign(transfer_message(coin.transfers[-1].signature, bob_public_key)),
    public_key=bob_public_key,
)

coin.transfers.append(alice_to_bob)
coin.validate()